In [ ]:
import py3dep
import geopandas as gpd
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pynhd as nhd
import flopy
import geopandas as gpd
from pynhd import NLDI, NHDPlusHR, WaterData
import contextily as ctx
import numpy as np

from flopy.utils import Raster
import os

import geopandas as gpd

In [ ]:
from xrspatial import zonal_stats
from rasterio import features
from rasterio.enums import MergeAlg
from rasterio.plot import show
import rasterio
import xarray as xr

use xarray zonal stats to calculate minimum elelvation for grid cells


https://pygis.io/docs/e_raster_rasterize.html

In [ ]:
import basic

In [ ]:
m = basic.load_model()

In [ ]:
x = m.modelgrid.get_xcellcenters_for_layer(1)
y = m.modelgrid.get_ycellcenters_for_layer(1)


In [ ]:
mgrid = m.modelgrid

In [ ]:

dst_crs = 'EPSG:2226'
raster_ws = os.path.join("GIS", "DEM")
raster_name = "output_be_DTM.tif"


rast = os.path.join(raster_ws, raster_name)
rio = Raster.load(rast)



In [ ]:
rio.bounds


In [ ]:


arr = rio.get_array(1)
idx = np.isfinite(arr)

vmin, vmax = arr[idx].min(), arr[idx].max()
vmin, vmax



In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1, aspect="equal")

ax = rio.plot(ax=ax, vmin=vmin, vmax=vmax)
plt.colorbar(ax.images[0], shrink=0.7);





In [ ]:
rio.bounds

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1, aspect="equal")

ax = rio.plot(ax=ax, vmin=vmin, vmax=vmax)
plt.colorbar(ax.images[0], shrink=0.7)
pmv = flopy.plot.PlotMapView(modelgrid=m.modelgrid)
pmv.plot_grid(ax=ax, lw=0.5, color="black");

https://pygis.io/docs/e_raster_rasterize.html

In [ ]:
grid = gpd.read_file("GIS/grid.shp")

In [ ]:

# Get list of geometries for all features in vector file
# geom = [shapes for shapes in grid.geometry]
# # create tuples of geometry, value pairs, where value is the attribute value you want to burn
# geom_value = ((geom,value) for geom, value in zip(vector.geometry, vector['id']))
geom = list((g.geometry, index) for index,g in grid.iterrows())
# Read in vector
vector = grid

# Open  raster
raster = rasterio.open(rast)

In [ ]:
# Rasterize vector using the shape and coordinate system of the raster
rasterized_grid = features.rasterize(geom,
                                out_shape = raster.shape,
                                fill = 0,
                                out = None,
                                transform = raster.transform,
                                all_touched = True,
                                default_value = 1,
                                dtype = None)

# Plot raster
fig, ax = plt.subplots(1, figsize = (10, 10))
show(rasterized_grid, ax = ax)
plt.gca().invert_yaxis()

In [ ]:
data = raster.read()[0,:,:]
data.shape

In [ ]:
rasterized_grid.shape

In [ ]:
xr_rast_grid = xr.DataArray(rasterized_grid)
xr_rast = xr.DataArray(data)

In [ ]:
zone = zonal_stats(xr_rast_grid, xr_rast)

In [ ]:
zone.head()

In [ ]:
zone_ras = zone.loc[:,'min'].values.reshape((m.dis.nrow, m.dis.ncol))

zone_ras = np.ma.array(zone_ras, mask = m.bas6.ibound.array[0]==0)

In [ ]:
zone_var = zone.loc[:,'std'].values.reshape((m.dis.nrow, m.dis.ncol))

zone_var = np.ma.array(zone_var, mask = m.bas6.ibound.array[0]==0)
# now to visualize using flopy and matplotlib
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1, aspect="equal")

pmv = flopy.plot.PlotMapView(modelgrid=m.modelgrid, ax=ax)
ax = pmv.plot_array(
    zone_var, masked_values=rio.nodatavals, cmap = 'terrain'
)

plt.colorbar(ax, shrink=0.7);
plt.title('Model Cell Variance')

In [ ]:


# now to visualize using flopy and matplotlib
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1, aspect="equal")
zone_ras = np.ma.array(zone_ras, mask = m.bas6.ibound.array[0]==0)
pmv = flopy.plot.PlotMapView(modelgrid=m.modelgrid, ax=ax)
cr = pmv.plot_array(
    zone_ras, masked_values=rio.nodatavals, cmap = 'terrain',vmax = 400
)

plt.colorbar(cr, shrink=0.7);

# sfr.plot(ax = ax, zorder = 100, edgecolor = 'k')


In [ ]:
sfr_file = r"C:\modeling\MirabelWohler\SFR_files\only_sfr_lines.shp"

sfr = gpd.read_file(sfr_file)

In [ ]:
import conda_scripts.arich_functions as af

In [ ]:
existing = np.loadtxt("RR_2022/inputs/lay_1_top.csv", delimiter = ',')
ex_gdf = af.array2rc(existing, 'elevation')
ex_gdf = pd.merge(ex_gdf, af.get_active_mod_df(m.modelgrid, m).loc[:,['i', 'j','geometry']], on = ['i', 'j'])

ex_gdf = gpd.GeoDataFrame(ex_gdf, geometry = 'geometry', crs = 2226)
ex_gdf.explore('elevation')

In [ ]:
plt.imshow(existing - zone_ras)
plt.colorbar( shrink=0.7);

In [ ]:
np.savetxt("GIS/lay_1_top.csv", zone_ras, delimiter=",", fmt ="%.3f")
np.savetxt("RR_2022/inputs/lay_1_top.csv", zone_ras, delimiter=",", fmt ="%.3f")

In [ ]:
# the following can be used to burn in stream cells

In [ ]:
sfr_file = r"GIS/SWR_Reaches.shp"

sfr = gpd.read_file(sfr_file)


In [ ]:
def plot_swr(sfr_filt):
    fig,ax = plt.subplots()
    
    ax.plot(sfr_filt.rno, sfr_filt.top)
    
    sfr_filt.loc[:,'top_burn'] = sfr_filt.set_index('rno').loc[:,'top'].rolling(20,min_periods = 1, center =False).min()
    sfr_filt.loc[:,'top_burn'] = sfr_filt.loc[:,'top_burn'].fillna(method = 'backfill')
    ax.plot(sfr_filt.rno, sfr_filt.top_burn)
    
    return sfr_filt
    
sfr_filt = plot_swr(sfr)

In [ ]:
sfr_filt.top_burn

In [ ]:
# burn in sfr elevations
burn = True

In [ ]:
a = np.array([[1,2],[3,4]])

display(a)

a[np.array([0,0]),np.array([0,1])]=-1

display(a)

In [ ]:

elev_array = np.copy(zone_ras)

plt.imshow(elev_array)

if burn:
    # elev_array[sfr.i.values, sfr.k.values] = sfr.strtop.values
    elevog = np.copy(elev_array)
    elev_array[sfr.i.values, sfr.j.values] = sfr.top_burn.values
    plt.figure()
    print('burning stream segments')
    im = plt.imshow(elev_array-elevog, interpolation = 'bilinear')
    # cbar = ax.figure.colorbar(im)
    plt.colorbar()
else:
    print('not burning stream segments')

In [ ]:
np.where(elev_array>2000)[1]

In [ ]:
np.savetxt("GIS/lay_1_top.csv", elev_array, delimiter=",", fmt ="%.1f")
np.savetxt("RR_2022/inputs/lay_1_top.csv", elev_array, delimiter=",", fmt ="%.1f")

In [ ]:
"max elevation: {}".format(np.max(elev_array))

In [ ]:
elev_array[1,67]

In [ ]:
zone_ras[1,67]

In [ ]:
plt.imshow(existing - elev_array)
plt.colorbar( shrink=0.7);

In [ ]:
# now to visualize using flopy and matplotlib
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1, aspect="equal")
elev_array_ma = np.ma.array(elev_array, mask = m.bas6.ibound.array[0]==0)
pmv = flopy.plot.PlotMapView(modelgrid=m.modelgrid, ax=ax)
cr = pmv.plot_array(
    elev_array_ma, masked_values=rio.nodatavals, cmap = 'terrain',vmax = 200
)

plt.colorbar(cr, shrink=0.7);

# sfr.plot(ax = ax, zorder = 100, edgecolor = 'k')